# Imports

In [200]:
import numpy as np
import pandas as pd

from prophet import Prophet
import time
from prophet.plot import plot_plotly, plot_components_plotly


In [ ]:
dollar = pd.read_csv("C:/Users/Luky-/Documents/Nuclio/DS market data/data_dsmarket/dollar_value_table.zip", index_col=0)
sales = pd.read_csv("C:/Users/Luky-/Documents/Nuclio/DS market data/data_dsmarket/item_sales.zip  ")
calendar = pd.read_csv("C:/Users/Luky-/Documents/Nuclio/DS market data/data_dsmarket/daily_calendar_with_events.csv")

# Data preparation

For a beta version of our model, we have decided to select only a some items to be predicted. We will select one store store (the one with most sales) and select the item that were sold the most in the last 30 days 

In [202]:
# From task 1 we know that the store with the most sales is tribeca, thus I will take the 30 last days of the df an look what items were the most sold in that time period
Tribeca = dollar[dollar.store=='Tribeca']

In [ ]:
calendar.tail(30)

,date,weekday,weekday_int,d,event
1883,2016-03-26,Saturday,1,d_1884,NaN
1884,2016-03-27,Sunday,2,d_1885,Easter
1885,2016-03-28,Monday,3,d_1886,NaN
1886,2016-03-29,Tuesday,4,d_1887,NaN
1887,2016-03-30,Wednesday,5,d_1888,NaN
1888,2016-03-31,Thursday,6,d_1889,NaN
1889,2016-04-01,Friday,7,d_1890,NaN
1890,2016-04-02,Saturday,1,d_1891,NaN
1891,2016-04-03,Sunday,2,d_1892,NaN
1892,2016-04-04,Monday,3,d_1893,NaN


In [ ]:
# Selecting the 30 last days
Tribeca = Tribeca[Tribeca.date >= "2016-03-26"]
# Grouping by items 
Tribeca = Tribeca[["id", "unit_sold"]].groupby(by="id").sum().sort_values(by='unit_sold',ascending=False)

In [ ]:
# I will select the 10 most sold Items in the last month
items_selected = Tribeca.sort_values(by="unit_sold", ascending= False).iloc[:10].index

In [ ]:
Tribeca

,unit_sold
id,
SUPERMARKET_3_090_NYC_3,3520
SUPERMARKET_3_586_NYC_3,2084
SUPERMARKET_3_120_NYC_3,2026
SUPERMARKET_3_252_NYC_3,1426
SUPERMARKET_3_282_NYC_3,1366
...,...
SUPERMARKET_1_120_NYC_3,0
HOME_&_GARDEN_2_498_NYC_3,0
HOME_&_GARDEN_1_151_NYC_3,0


In [ ]:
# making a df with only the selected items
items_selected_indx = dollar[dollar.id.isin(items_selected)].index

In [ ]:
df_list = []

In [ ]:
def prophet_df(dataframe, items_selected):
    df_list = []
    for item in items_selected:       
        series = dataframe.loc[dataframe['id'] == item][['id', 'date', 'unit_sold']].set_index('id')
        ph_df = series.rename(columns={'date': 'ds', 'unit_sold':'y'})
        df_list.append(ph_df)
        
    return df_list     

In [ ]:
df_list = prophet_df(dollar, items_selected)
df_list

[                                 ds    y
 id                                      
 SUPERMARKET_3_090_NYC_3  2011-01-29  108
 SUPERMARKET_3_090_NYC_3  2011-01-30  132
 SUPERMARKET_3_090_NYC_3  2011-01-31  102
 SUPERMARKET_3_090_NYC_3  2011-02-01  120
 SUPERMARKET_3_090_NYC_3  2011-02-02  106
 ...                             ...  ...
 SUPERMARKET_3_090_NYC_3  2016-04-20   88
 SUPERMARKET_3_090_NYC_3  2016-04-21   77
 SUPERMARKET_3_090_NYC_3  2016-04-22  141
 SUPERMARKET_3_090_NYC_3  2016-04-23  139
 SUPERMARKET_3_090_NYC_3  2016-04-24  130
 
 [1913 rows x 2 columns],
                                  ds   y
 id                                     
 SUPERMARKET_3_586_NYC_3  2011-01-29  56
 SUPERMARKET_3_586_NYC_3  2011-01-30  55
 SUPERMARKET_3_586_NYC_3  2011-01-31  45
 SUPERMARKET_3_586_NYC_3  2011-02-01  57
 SUPERMARKET_3_586_NYC_3  2011-02-02  54
 ...                             ...  ..
 SUPERMARKET_3_586_NYC_3  2016-04-20  70
 SUPERMARKET_3_586_NYC_3  2016-04-21  48
 SUPERMARKET_3_5

## Modelisation

In [ ]:
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
param_grid= {  
    'changepoint_prior_scale': [0.01,0.1],
    'seasonality_prior_scale': [0.01, 0.1],}
# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v) for v in itertools.product(*param_grid.values()))]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    for x in range(len(df_list)):
        m = Prophet(**params).fit(df_list[x])  # Fit model with given params
        m.add_country_holidays('US')
        df_cv = cross_validation(m, horizon='30 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

TypeError: 'dict' object is not callable

In [ ]:
def run_prophet(timeserie):
    model = Prophet()   
    model.add_country_holidays(country_name='US')
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=28)
    forecast = model.predict(forecast)
    forecast["id"] =  product.index[0]

    return forecast

In [ ]:
start_time = time.time()
result = list(map(run_prophet,df_list))
print("--- %s seconds ---" % (time.time() - start_time))

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


--- 62.944268465042114 seconds ---


In [ ]:
result

[             ds      trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
 0    2011-01-29 -24.079083  -65.057747  147.987952   -24.079083   -24.079083   
 1    2011-01-30 -23.494659  -93.074515  122.474165   -23.494659   -23.494659   
 2    2011-01-31 -22.910236 -118.476059   86.228997   -22.910236   -22.910236   
 3    2011-02-01 -22.325813 -120.431620   77.208087   -22.325813   -22.325813   
 4    2011-02-02 -21.741390 -127.247899   82.049398   -21.741390   -21.741390   
 ...         ...        ...         ...         ...          ...          ...   
 1936 2016-05-18  95.417683  -88.574927  110.524375    95.273455    95.582287   
 1937 2016-05-19  95.427332  -82.792767  120.911529    95.243380    95.620537   
 1938 2016-05-20  95.436981  -51.254571  155.239373    95.222303    95.672190   
 1939 2016-05-21  95.446630  -27.487090  177.261285    95.205145    95.716184   
 1940 2016-05-22  95.456279  -69.392083  138.769070    95.183090    95.794308   
 
       Christmas Day  Chri

## Evaluating the results

In [ ]:
plot_plotly(model,result)

The model is quiet accurate appart in july were there is a peek that the model doesn't catch

In [ ]:
#Saving a version of the predictions
predictions.to_csv("Tribeca_products_predictions.csv")

In [ ]:
#Making a consolidated df with real available values and predictions
Consolidated_predictions = pd.concat([predictions.set_index('id').rename(columns={"yhat":"y"}),prophet_df])
Consolidated_predictions['ds'] = pd.to_datetime(Consolidated_predictions['ds'])
Consolidated_predictions.sort_values(by='ds', inplace=True)

In [ ]:
Consolidated_predictions.to_csv("Consolidated_predictions.csv")

# Visualitation